In [ ]:
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import joblib
import time

from sklearn.linear_model import LogisticRegression
from keras.models import load_model
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import EarlyStopping
from tensorflow import keras
from keras.layers import LSTM,Dense,Dropout,BatchNormalization,LayerNormalization, InputLayer
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score

In [ ]:
# read in the processed data
All_Games = pd.read_csv("All_Matches.csv")

In [ ]:
# one hot encode the categorical columns

encoder = OneHotEncoder()
encoder_df = pd.DataFrame(encoder.fit_transform(All_Games[["Rank","Top_Current_Champ", "Jg_Current_Champ", "Mid_Current_Champ", "Bot_Current_Champ", "Sup_Current_Champ","Top_Enemy_Champ", "Jg_Enemy_Champ", "Mid_Enemy_Champ", "Bot_Enemy_Champ", "Sup_Enemy_Champ"]]).toarray())

In [ ]:
encoder_df

In [ ]:
# drop the categorical columns and join the one hot encoded version

All_Games = All_Games.drop(["Rank","Top_Current_Champ", "Jg_Current_Champ", "Mid_Current_Champ", "Bot_Current_Champ", "Sup_Current_Champ","Top_Enemy_Champ", "Jg_Enemy_Champ", "Mid_Enemy_Champ", "Bot_Enemy_Champ", "Sup_Enemy_Champ"],axis=1)
All_Games = pd.DataFrame(np.hstack([All_Games, encoder_df]))

In [ ]:
All_Games

In [ ]:
#All_Games = shuffle(All_Games)

In [ ]:
X = All_Games.iloc[:,1:]
y = All_Games.iloc[:,0]

In [ ]:
# train_ratio = 0.80
# test_ratio = 0.10
# validation_ratio = 0.10

# X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_ratio)

# X_train, X_valid, Y_train, Y_valid = train_test_split(X_train, Y_train, test_size=validation_ratio/(train_ratio+test_ratio))

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, y, test_size=.20, shuffle=True,random_state=5)

In [ ]:
#split the 80-20 to 80-10-10

SplitX = X_valid
SplitY = Y_valid

X_valid = SplitX.iloc[:1500]
Y_valid = SplitY.iloc[:1500]

X_test = SplitX.iloc[1500:]
Y_test = SplitY.iloc[1500:]

In [ ]:
#scale the data using minmax

sc = MinMaxScaler()

X_train = sc.fit_transform(X_train)
X_valid = sc.transform(X_valid)
X_test = sc.transform(X_test)

In [ ]:
#create the model

model_one_hot = keras.Sequential() 

model_one_hot.add(InputLayer(input_shape=X_train.shape[1]))
model_one_hot.add(Dense(128, activation='relu'))  

model_one_hot.add(Dense(1, activation='sigmoid')) 

model_one_hot.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), metrics=['acc'])

model_one_hot.summary()

In [ ]:
# early stopping

es = EarlyStopping(monitor='val_acc', mode='max', patience=10, restore_best_weights=True)

In [ ]:
# fit with early stopping

with tf.device('/GPU:0'):
    fit_model_one_hot = model_one_hot.fit(X_train, Y_train, validation_data=(X_valid,Y_valid), epochs=250, callbacks=es)

In [ ]:
# print loss and score on testing

score = model_one_hot.evaluate(X_test, Y_test,verbose=0)
#score = model_one_hot.evaluate(X_valid, Y_valid,verbose=0)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

In [ ]:
# save the model

model_one_hot.save("KerasModel.h5")

In [ ]:
# plit graphs

plt.plot(fit_model_one_hot.history['acc'])
plt.plot(fit_model_one_hot.history['val_acc'])
plt.legend(['accuracy','validation accuracy'])

In [ ]:
# save the scaler and encoder to be used to for predictions later on the website

joblib.dump(sc, 'MinMaxScaler.gz')
joblib.dump(encoder,'OneHotEncoder.joblib')

In [ ]:
# load the scaler and encdoer and transform the prediction dataframe/csv
# modified in the website file

def ProcessLiveData(data):
    min_max_scaler = joblib.load('MinMaxScaler.gz')
    one_hot_encoder = joblib.load('OneHotEncoder.joblib')
    encoder_lobby = pd.DataFrame(one_hot_encoder.transform(data[["Rank","Top_Current_Champ", "Jg_Current_Champ", "Mid_Current_Champ", "Bot_Current_Champ", "Sup_Current_Champ","Top_Enemy_Champ", "Jg_Enemy_Champ", "Mid_Enemy_Champ", "Bot_Enemy_Champ", "Sup_Enemy_Champ"]]).toarray())
    data = data.join(encoder_lobby)
    data = data.drop(["Rank","Top_Current_Champ", "Jg_Current_Champ", "Mid_Current_Champ", "Bot_Current_Champ", "Sup_Current_Champ","Top_Enemy_Champ", "Jg_Enemy_Champ", "Mid_Enemy_Champ", "Bot_Enemy_Champ", "Sup_Enemy_Champ"],axis=1)
    data["Side"] = data["Side"].astype('category')
    data["Side"] = data["Side"].cat.codes
    data = min_max_scaler.transform(data)
    return data

In [ ]:
# load the model call the process data function and then use that to output a prediction

def Predict():
    model = load_model('KerasModel.h5')
    lobby =  pd.read_csv("lobby.csv")
    NewPredict = ProcessLiveData(lobby)
    return model.predict(NewPredict)[0,0]

In [ ]:
# holds the prediction

prediction = Predict()

In [ ]:
# output the prediction

prediction

In [ ]:
# train multiple times

accuracies = []

for i in range(15):
    
    model_one_hot = keras.Sequential() 
    model_one_hot.add(InputLayer(input_shape=X_train.shape[1]))
    model_one_hot.add(Dense(128, activation='relu'))  
    model_one_hot.add(Dense(1, activation='sigmoid')) 
    model_one_hot.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.01), metrics=['acc'])

    fit_model_one_hot = model_one_hot.fit(X_train, Y_train, validation_data=(X_valid,Y_valid), epochs=250, callbacks=es)
    score = model_one_hot.evaluate(X_test, Y_test,verbose=0)
    accuracies.append(score[1])

In [ ]:
# calculate the mean and standard deviation

accuracies_percent = [element * 100 for element in accuracies]

print("The original list : " + str(accuracies_percent))
print("mean : ",sum(accuracies_percent) / len(accuracies_percent))
# Standard deviation of list
# Using sum() + list comprehension
mean = sum(accuracies_percent) / len(accuracies_percent)
variance = sum([((x - mean) ** 2) for x in accuracies_percent]) / len(accuracies_percent)
res = variance ** 0.5
  
# Printing result
print("Standard deviation of sample is : " + str(res))